# STEP 1
## Data collection
## Verify that the SQL database works and we can extract assay information

In [2]:
import sqlite3
import csv
import pandas as pd

In [3]:
conn = sqlite3.connect('sabs_moonshot.db')

In [4]:
"""
Join assasy and compounds table, and output assay information alongside smiles string
"""

df = pd.read_sql_query("""
SELECT *
FROM assays
    INNER JOIN compounds ON compounds.id = assays.compound_id
WHERE compounds.assayed = TRUE ORDER BY smiles;
""",
conn)
df[:10]

,id,compound_id,r_inhibition_at_20_uM,r_inhibition_at_50_uM,r_avg_IC50,r_curve_IC50,r_max_inhibition_reading,r_min_inhibition_reading,r_hill_slope,r_R2,...,f_hill_slope,f_R2,relative_solubility_at_20_uM,relative_solubility_at_100_uM,trypsin_IC50,id,smiles,MW,NMR_std_ratio,assayed
0,STE-KUL-d79e3d6a-5,STE-KUL-d79e3d6a-5,,,,None,,,,,...,"[{'value': 0.915466976259181}, {'value': 0.964...","[{'value': 0.9863264317351539}, {'value': 0.98...",,,,STE-KUL-d79e3d6a-5,C#CCCCC(=O)N[C@@H](CC)C(=O)N[C@H](C(=O)N[C@@H]...,725.44,,1
1,STE-KUL-d79e3d6a-4,STE-KUL-d79e3d6a-4,,,,None,,,,,...,"[{'value': 0.8347410783476827}, {'value': 1.10...","[{'value': 0.9860619683158497}, {'value': 0.96...",,,,STE-KUL-d79e3d6a-4,C#CCCCC(=O)N[C@@H](CC)C(=O)N[C@H](C(=O)N[C@@H]...,697.41,,1
2,STE-KUL-d79e3d6a-2,STE-KUL-d79e3d6a-2,,,,None,,,,,...,"[{'value': 1.590711677149666}, {'value': 1.401...","[{'value': 0.8755085396715557}, {'value': 0.99...",,,,STE-KUL-d79e3d6a-2,C#CCCCC(=O)N[C@H](C(=O)N[C@H](CC(C)C)C(=O)NN(C...,535.3,,1
3,STE-KUL-d79e3d6a-1,STE-KUL-d79e3d6a-1,,,,None,,,,,...,"[{'value': 1.5319145456767893}, {'value': 1.59...","[{'value': 0.9940096304686957}, {'value': 0.99...",,,,STE-KUL-d79e3d6a-1,C#CCCCC(=O)N[C@H](C(=O)N[C@H](CC(C)C)C(=O)NN(C...,565.31,,1
4,STE-KUL-d79e3d6a-3,STE-KUL-d79e3d6a-3,,,,None,,,,,...,"[{'value': 2.2529412177815855}, {'value': 1.20...","[{'value': 0.9906857511930752}, {'value': 0.94...",,,,STE-KUL-d79e3d6a-3,C#CCCCC(=O)N[C@H](C(=O)N[C@H](CC(C)C)C(=O)NN(C...,499.26,,1
5,PET-UNK-c9c1e0d8-2,PET-UNK-c9c1e0d8-2,,,,None,[nan],[nan],[nan],[nan],...,[{'value': 1.159785525866891}],[{'value': 0.9778869511548216}],,,,PET-UNK-c9c1e0d8-2,C#Cc1cccc(CC(=O)Nc2cncc3ccccc23)c1,286.11,,1
6,RED-RED-10c9212c-31,RED-RED-10c9212c-31,,,,None,,,,,...,[nan],[nan],,,,RED-RED-10c9212c-31,C/C=C/C=C/C(=O)N1CC=C(c2c[nH]c3ncccc23)CC1,293.15,,1
7,RED-RED-10c9212c-23,RED-RED-10c9212c-23,,,,None,,,,,...,[nan],[nan],,,,RED-RED-10c9212c-23,C1=C(c2c[nH]c3ccccc23)CCN(Cc2ccno2)C1,279.14,,1
8,MAT-POS-090737b9-1,MAT-POS-090737b9-1,,,,None,,,,,...,"[{'value': 0.6880723926589011}, {'value': 1.47...","[{'value': 0.6154567663231494}, {'value': 0.97...",,,,MAT-POS-090737b9-1,C=CC(=O)N(C(=O)C1CCOc2ccc(Cl)cc21)c1cncc2ccccc12,392.09,,1
9,MAT-POS-e69ad64a-2,MAT-POS-e69ad64a-2,,,,None,,,,,...,"[{'value': 1.1489910631267832}, {'value': 1.17...","[{'value': 0.9869283640406017}, {'value': 0.99...",,,,MAT-POS-e69ad64a-2,C=CC(=O)N(C(=O)C1COc2ccc(Cl)cc21)c1cncc2ccccc12,378.08,,1


In [5]:
"""
Lookup all assays for a specific smiles string
"""

df = pd.read_sql_query("""
SELECT *
FROM assays
    INNER JOIN compounds ON compounds.id = assays.compound_id
WHERE compounds.assayed = TRUE AND compounds.smiles = "C/C=C/C=C/C(=O)N1CC=C(c2c[nH]c3ncccc23)CC1";
""",
conn)
df[:10]

,id,compound_id,r_inhibition_at_20_uM,r_inhibition_at_50_uM,r_avg_IC50,r_curve_IC50,r_max_inhibition_reading,r_min_inhibition_reading,r_hill_slope,r_R2,...,f_hill_slope,f_R2,relative_solubility_at_20_uM,relative_solubility_at_100_uM,trypsin_IC50,id,smiles,MW,NMR_std_ratio,assayed
0,RED-RED-10c9212c-31,RED-RED-10c9212c-31,,,,None,,,,,...,[nan],[nan],,,,RED-RED-10c9212c-31,C/C=C/C=C/C(=O)N1CC=C(c2c[nH]c3ncccc23)CC1,293.15,,1


In [6]:
data = pd.read_sql_query(
    """
    SELECT
        compounds.smiles, compounds.MW, compounds.NMR_std_ratio,
        r_avg_IC50, f_avg_IC50, trypsin_IC50
    FROM assays
    INNER JOIN compounds ON compounds.id = assays.compound_id
    WHERE compounds.assayed = TRUE;
    """,
    conn
)
data

,smiles,MW,NMR_std_ratio,r_avg_IC50,f_avg_IC50,trypsin_IC50
0,N#Cc1cccc(NC(=O)Nc2cncc(N)c2)c1,253.265,,,54.273,99
1,N#Cc1cccc(NC(=O)Nc2c[nH]c3ncccc23)c1,277.287,,,63.5549,99
2,N#Cc1cccc(NC(=O)Cc2cncc3ccccc23)c1,287.322,,14.6411,26.6801,
3,O=C(Cc1cncc2ccccc12)Nc1ccccc1,262.312,,45.0775,57.4697,99
4,N#Cc1cc(Cl)cc(NC(=O)Nc2cccnc2)c1,272.695,,,99.01,99
...,...,...,...,...,...,...
2433,Cc1cc(C(=O)N2Cc3ccc(Cl)cc3C3(CCN(c4cncc5ccccc4...,485.16,,,0.463982,
2434,CCn1nccc1C(=O)N1Cc2ccc(Cl)cc2C2(CCN(c3cncc4ccc...,485.16,,,0.297012,
2435,CNC(=O)CN1C[C@](C)(C(=O)Nc2cncc3ccccc23)c2cc(C...,472.1,,,,
2436,CNC(=O)CN1C[C@](C)(C(=O)Nc2cncc3ccccc23)c2cc(C...,472.1,,,,


In [9]:
data = pd.read_sql_query(
    """
    SELECT COUNT(*)
    FROM compounds
    INNER JOIN assays ON compounds.id = assays.compound_id
    WHERE compounds.assayed = TRUE AND assays.r_avg_IC50 != "";
    """,
    conn
)
data

,COUNT(*)
0,876


In [10]:
data = pd.read_sql_query(
    """
    SELECT COUNT(*)
    FROM compounds
    INNER JOIN assays ON compounds.id = assays.compound_id
    WHERE compounds.assayed = TRUE AND assays.f_avg_IC50 != "";
    """,
    conn
)
data

,COUNT(*)
0,2220
